## [Transformer와 비교해 변경이 필요한 부분]
- Positional Encoding 대신 Positional Embedding 사용
- 인코더 삭제
    - 인코더 디코더 어텐션 삭제
- input 데이터 형식 변경


In [67]:
import tensorflow

print(tensorflow.__version__)

2.13.1


In [68]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import matplotlib.pyplot as plt

### 포지셔널 행렬 구현 -> Positional Embedding 으로


In [69]:
# 포지셔널 임베딩 레이어

class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_seq_len, d_model):
        super().__init__()
        self.max_seq_len = max_seq_len
        self.d_model = d_model
        self.embedding = tf.keras.layers.Embedding(
            input_dim=max_seq_len, 
            output_dim=d_model,
            input_length=max_seq_len
        )

    def call(self, x):
        positions = tf.range(start=0, limit=tf.shape(x)[1], delta=1)
        positions = tf.expand_dims(positions, axis=0)
        positions = tf.broadcast_to(positions, [tf.shape(x)[0], tf.shape(x)[1]])
        return self.embedding(positions)

### Scaled dot product attention

In [70]:
# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
  # 어텐션 가중치는 Q와 K의 닷 프로덕트
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
  if mask is not None:
    logits += (mask * -1e9)

  # softmax적용
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
  output = tf.matmul(attention_weights, value)
  return output

### Multi-head attention

In [71]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # Q, K, V에 각각 Dense를 적용합니다
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value) 

    # 병렬 연산을 위한 머리를 여러 개 만듭니다
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 스케일드 닷 프로덕트 어텐션 함수
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 최종 결과에도 Dense를 한 번 더 적용합니다
    outputs = self.dense(concat_attention)

    return outputs

### Padding masking

In [72]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

### Look ahead masking

In [73]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

### Encoder (삭제 X)

In [74]:
# # 인코더 하나의 레이어를 함수로 구현.
# # 이 하나의 레이어 안에는 두 개의 서브 레이어가 존재합니다.
# def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
#   inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

#   # 패딩 마스크 사용
#   padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

#   # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
#   attention = MultiHeadAttention(
#       d_model, num_heads, name="attention")({
#           'query': inputs,
#           'key': inputs,
#           'value': inputs,
#           'mask': padding_mask
#       })

#   # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
#   attention = tf.keras.layers.Dropout(rate=dropout)(attention)
#   attention = tf.keras.layers.LayerNormalization(
#       epsilon=1e-6)(inputs + attention)

#   # 두 번째 서브 레이어 : 2개의 완전연결층
#   outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
#   outputs = tf.keras.layers.Dense(units=d_model)(outputs)

#   # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
#   outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
#   outputs = tf.keras.layers.LayerNormalization(
#       epsilon=1e-6)(attention + outputs)

#   return tf.keras.Model(
#       inputs=[inputs, padding_mask], outputs=outputs, name=name)


In [75]:
# def encoder(vocab_size,
#             num_layers,
#             units,
#             d_model,
#             num_heads,
#             dropout,
#             name="encoder"):
#   inputs = tf.keras.Input(shape=(None,), name="inputs")

#   # 패딩 마스크 사용
#   padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

#   # 임베딩 레이어
#   embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
#   embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

#   # 포지셔널 인코딩
#   embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

#   outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

#   # num_layers만큼 쌓아올린 인코더의 층.
#   for i in range(num_layers):
#     outputs = encoder_layer(
#         units=units,
#         d_model=d_model,
#         num_heads=num_heads,
#         dropout=dropout,
#         name="encoder_layer_{}".format(i),
#     )([outputs, padding_mask])

#   return tf.keras.Model(
#       inputs=[inputs, padding_mask], outputs=outputs, name=name)


### Decoder (Encoder-decoder attention 제거)


In [76]:
# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })

  # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)
  
# Encoder와 연결되는 Encoder-decoder attention은 제거

#   # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
#   attention2 = MultiHeadAttention(
#       d_model, num_heads, name="attention_2")(inputs={
#           'query': attention1,
#           'key': enc_outputs,
#           'value': enc_outputs,
#           'mask': padding_mask
#       })

#   # 마스크드 멀티 헤드 어텐션의 결과는
#   # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
#   attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
#   attention2 = tf.keras.layers.LayerNormalization(
#       epsilon=1e-6)(attention2 + attention1)

  # 세 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention1) # 두 번째 attention 층이 없어졌으므로 첫 번째 attention층 결과와와 연결
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs) # 기존 outputs + attention2 에서 attention2를 지움움

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)


In [77]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  # Dropout이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

---

### 데이터 받아오기

In [78]:
import pandas as pd

# 데이터 불러오기
df = pd.read_csv("data/ChatbotData.csv")

# 각각 저장
df["Q"].to_csv("data/question.txt", index=False, header=False)
df["A"].to_csv("data/answer.txt", index=False, header=False)
df["label"].to_csv("data/label.txt", index=False, header=False)


In [79]:
base_path = os.path.abspath("data")

# 저장된 파일 경로 출력
question_path = os.path.join(base_path, "question.txt")
answer_path = os.path.join(base_path, "answer.txt")
label_path = os.path.join(base_path, "label.txt")

In [80]:
print(len(df))

11823


데이터가 그리 크지는 않아서 따로 샘플 최대 개수를 설정하지는 않겠습니다

### 전처리 함수
- 질문과 답변을 하나의 시퀀스로 결합
    - "Q + Delim + A" 형태로 변환
- 시퀀스 앞뒤에 특수 토큰 추가
    - (Start) + Q + Delim + A + (Extract)
- Tokenization

In [109]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import GPT2Tokenizer

# 토크나이저 로드 (GPT2 토크나이저 사용)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # PAD 토큰 설정

# 데이터 로드
df = pd.read_csv('data/ChatbotData.csv')  # CSV 파일 경로를 지정해주세요

# Q와 A를 결합하여 하나의 시퀀스로 만들기
text_samples = []
for q, a in zip(df['Q'], df['A']):
    # Q와 A를 구분자로 구분하여 결합
    combined_text = f"{q} <|sep|> {a}"
    text_samples.append(combined_text)

def encode_and_pad(sentences, max_length=50):
    """
    문장을 토큰화하고 정수 인코딩한 후, 패딩을 적용하는 함수
    """
    encoded = tokenizer(sentences, 
                       padding="max_length",
                       truncation=True,
                       max_length=max_length,
                       return_tensors="tf")  # TensorFlow tensor로 반환
    return encoded.input_ids

# 토큰화 + 정수 인코딩 + 패딩 적용
max_seq_len = 50
input_ids = encode_and_pad(text_samples, max_length=max_seq_len)

# 데이터셋 생성
def create_gpt_dataset(input_ids, batch_size=16):
    """
    GPT 학습을 위한 데이터셋 생성
    입력: 현재 토큰
    출력: 다음 토큰
    """
    # 입력과 타겟 생성
    def create_input_target(sequence):
        input_seq = sequence[:-1]  # 마지막 토큰을 제외한 모든 토큰
        target_seq = sequence[1:]  # 첫 토큰을 제외한 모든 토큰
        return input_seq, target_seq

    dataset = tf.data.Dataset.from_tensor_slices(tf.cast(input_ids, tf.int32))
    dataset = dataset.map(create_input_target)
    dataset = dataset.shuffle(10000)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

# 데이터셋 생성
dataset = create_gpt_dataset(input_ids)

# 단어장 크기 확인
vocab_size = len(tokenizer)
print(f"단어장 크기 (Vocab Size): {vocab_size}")

# 데이터셋 확인
for input_example, target_example in dataset.take(1):
    print(f"입력 shape: {input_example.shape}")
    print(f"타겟 shape: {target_example.shape}")

단어장 크기 (Vocab Size): 50257
입력 shape: (16, 49)
타겟 shape: (16, 49)


### Dataloader 변환

In [110]:
import tensorflow as tf

def create_gpt_dataset(input_ids, batch_size=16):
    """
    GPT 학습을 위한 데이터셋 생성
    (입력: sequence[:, :-1], 타겟: sequence[:, 1:])
    """
    def map_input_target(sequence):
        # 마지막 토큰 제외한 것이 입력, 첫 번째 토큰 제외한 것이 타겟
        return sequence[:-1], sequence[1:]

    dataset = tf.data.Dataset.from_tensor_slices(tf.cast(input_ids, tf.int32))

    # (입력, 타겟) 쌍으로 만들기
    dataset = dataset.map(map_input_target)

    # 섞기 + 배치 + Prefetch
    dataset = dataset.shuffle(buffer_size=len(input_ids))
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return dataset

# 예시
batch_size = 16
dataset = create_gpt_dataset(input_ids, batch_size=batch_size)

for input_batch, target_batch in dataset.take(1):
    print("입력 배치 크기:", input_batch.shape)
    print("타겟 배치 크기:", target_batch.shape)
    print("입력 배치 예시(첫 샘플):", input_batch[0].numpy())
    print("타겟 배치 예시(첫 샘플):", target_batch[0].numpy())

입력 배치 크기: (16, 49)
타겟 배치 크기: (16, 49)
입력 배치 예시(첫 샘플): [  169   227   242   167   254   230   167   117   226   168   254   226
 31619   111   120   220   166   110   234 23821   245   228   168   244
   112  1279    91   325    79    91    29 23821   254   222   167   252
   239 23821   251   112   168   243   120   166   116   108   220 47991
   112]
타겟 배치 예시(첫 샘플): [  227   242   167   254   230   167   117   226   168   254   226 31619
   111   120   220   166   110   234 23821   245   228   168   244   112
  1279    91   325    79    91    29 23821   254   222   167   252   239
 23821   251   112   168   243   120   166   116   108   220 47991   112
   168]


In [111]:
# 최종적으로 model.fit()에 넣을 dataset
for sample in dataset.take(1):
    print(len(sample))  # 튜플 길이 확인 → 2여야 (입력, 타겟)임

    # (입력, 타겟)이라면,
    # sample[0]: 입력 텐서, sample[1]: 타겟 텐서
    print(f"입력 shape: {sample[0].shape}")
    print(f"타겟 shape: {sample[1].shape}")


2
입력 shape: (16, 49)
타겟 shape: (16, 49)


### 교사 강요 사용 (안 함)

In [83]:
# BATCH_SIZE = 64
# BUFFER_SIZE = 20000

# # 디코더는 이전의 target을 다음의 input으로 사용합니다.
# # 이에 따라 outputs에서는 START_TOKEN을 제거하겠습니다.
# dataset = tf.data.Dataset.from_tensor_slices((
#     {
#         'inputs': questions,
#         'dec_inputs': answers[:, :-1]
#     },
#     {
#         'outputs': answers[:, 1:]
#     },
# ))

# dataset = dataset.cache()
# dataset = dataset.shuffle(BUFFER_SIZE)
# dataset = dataset.batch(BATCH_SIZE)
# dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
# print("슝=3")

## 모델 정의 및 학습


### GPT 모델 정의

In [112]:
class TransformerDecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout_rate=0.1):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(d_ff, activation='relu'),  # Feed-Forward Network
            tf.keras.layers.Dense(d_model)  # 차원 축소
        ])
        self.ln1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.ln2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x, mask, training):
        """
        x: (batch_size, seq_len, d_model)
        mask: Look-Ahead Mask
        training: 학습 여부
        """
        # 1️. Self-Attention + Residual Connection
        attn_output = self.mha(x, x, attention_mask=mask)  # Self-Attention
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.ln1(x + attn_output)  # Residual + LayerNorm

        # 2️. FFN + Residual Connection
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.ln2(out1 + ffn_output)  # Residual + LayerNorm

        return out2


In [113]:
class GPT1(tf.keras.Model):
    def __init__(self, vocab_size, max_seq_len, d_model, num_layers, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.token_embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=d_model)
        self.positional_embedding = PositionalEmbedding(max_seq_len, d_model)

        self.transformer_blocks = [
            TransformerDecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)
        ]

        self.ln_f = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.fc_out = tf.keras.layers.Dense(vocab_size, activation='softmax')  # 최종 출력

    def call(self, x, training=False):
        """
        x: (batch_size, seq_len)
        """
        seq_len = tf.shape(x)[1]

        # 패딩 마스크 및 룩 어헤드 마스크 생성
        padding_mask = create_padding_mask(x)
        look_ahead_mask = create_look_ahead_mask(x)

        # 임베딩 적용 ('토큰 + 포지션 임베딩'을 입력으로 사용)
        tok_emb = self.token_embedding(x)  # (batch_size, seq_len, d_model)
        pos_emb = self.positional_embedding(x)  # (1, seq_len, d_model)
        x = tok_emb + pos_emb  # 두 개 더해서 최종 입력 만듦

        # Transformer 블록 적용
        for layer in self.transformer_blocks:
            x = layer(x, look_ahead_mask, training)  # 마스킹 적용

        x = self.ln_f(x)  # 마지막 LayerNorm
        logits = self.fc_out(x)  # (batch_size, seq_len, vocab_size)
        return logits


### 모델 생성

In [114]:
# 하이퍼파라미터 설정
vocab_size = 50257  # 어휘 사전 크기
num_layers = 12  # 디코더 레이어 개수
d_model = 768  # 모델 차원 (GPT-1 기준)
num_heads = 12  # 멀티헤드 어텐션 개수
d_ff = 3072  # FFN 차원 (GPT-1 논문 기준)
dropout = 0.1  # 드롭아웃 비율
max_seq_len = 50  # 최대 문장 길이

# GPT 모델 인스턴스 생성
model = GPT1(
    vocab_size=vocab_size,
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    d_ff=d_ff,
    dropout=dropout,
    max_seq_len=max_seq_len,
)

# 모델을 빌드 (입력 shape 명시)
model.build(input_shape=(None, max_seq_len))

# 모델 구조 출력
model.summary()

Model: "gpt1_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    multiple                  38597376  
                                                                 
 positional_embedding_5 (Po  multiple                  38400     
 sitionalEmbedding)                                              
                                                                 
 transformer_decoder_layer_  multiple                  33065472  
 60 (TransformerDecoderLaye                                      
 r)                                                              
                                                                 
 transformer_decoder_layer_  multiple                  33065472  
 61 (TransformerDecoderLaye                                      
 r)                                                              
                                                            

In [97]:
# 개별 레이어의 파라미터 수 확인
for layer in model.layers:
    print(f"Layer: {layer.name}, Parameters: {layer.count_params()}")

# 어휘 크기 확인
print(f"Vocab Size: {vocab_size}")

Layer: embedding_6, Parameters: 38597376
Layer: positional_embedding_3, Parameters: 38400
Layer: transformer_decoder_layer_36, Parameters: 33065472
Layer: transformer_decoder_layer_37, Parameters: 33065472
Layer: transformer_decoder_layer_38, Parameters: 33065472
Layer: transformer_decoder_layer_39, Parameters: 33065472
Layer: transformer_decoder_layer_40, Parameters: 33065472
Layer: transformer_decoder_layer_41, Parameters: 33065472
Layer: transformer_decoder_layer_42, Parameters: 33065472
Layer: transformer_decoder_layer_43, Parameters: 33065472
Layer: transformer_decoder_layer_44, Parameters: 33065472
Layer: transformer_decoder_layer_45, Parameters: 33065472
Layer: transformer_decoder_layer_46, Parameters: 33065472
Layer: transformer_decoder_layer_47, Parameters: 33065472
Layer: layer_normalization_99, Parameters: 1536
Layer: dense_99, Parameters: 38647633
Vocab Size: 50257


### 커스텀 된 학습률 - 뭔가 오류가 계속 나서 우선 안 쓰는 걸로..

In [ ]:
# class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

#   def __init__(self, d_model, warmup_steps=4000):
#     super(CustomSchedule, self).__init__()

#     self.d_model = d_model
#     self.d_model = tf.cast(self.d_model, tf.float32)

#     self.warmup_steps = warmup_steps

#   def __call__(self, step):
#     arg1 = tf.math.rsqrt(step, tf.float32)
#     arg2 = step * (self.warmup_steps**-1.5)

#     return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


In [ ]:
# sample_learning_rate = CustomSchedule(d_model=128)

# plt.plot(sample_learning_rate(tf.range(200000, dtype=tf.float32)))
# plt.ylabel("Learning Rate")
# plt.xlabel("Train Step")

### 손실 함수

In [ ]:
# # 손실 함수 및 옵티마이저 설정
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)  # Softmax 적용됨
# learning_rate = CustomSchedule(float(d_model))
# optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# # 모델 컴파일
# model.compile(optimizer=optimizer, loss=loss_fn)

### 모델 컴파일

In [ ]:
# learning_rate = CustomSchedule(d_model)

# optimizer = tf.keras.optimizers.Adam(
#     learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# def accuracy(y_true, y_pred):
#   y_true = tf.reshape(y_true, shape=(-1, max_seq_len - 1))
#   return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

# model.compile(optimizer=optimizer, loss=loss_fn, metrics=[accuracy])


In [115]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4, beta_1=0.9, beta_2=0.98, epsilon=1e-9),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [116]:
# 데이터셋 확인
for sample in dataset.take(1):
    print(f"입력 shape: {sample[0].shape}")
    print(f"타겟 shape: {sample[1].shape}")
    print(f"입력 샘플: {sample[0].numpy()}")
    print(f"타겟 샘플: {sample[1].numpy()}")

입력 shape: (16, 49)
타겟 shape: (16, 49)
입력 샘플: [[  169   247   242   168   252    98 23821    95   222 23821   246   230
    168   223   246   166   110   234   220 47991   246   166   111   254
  23821   233   114 46695    97  1279    91   325    79    91    29   220
  47991   246 46695    97   167   111   112   167   102   112 31619   232
    246]
 [  168   224   112   168   108   238   220   166   109   108   220   166
    108   247   168   243   226  1279    91   325    79    91    29   220
    166   116   108   167   114   226 35975   120   166   118   120   168
    245   238   168   248   242    13 50256 50256 50256 50256 50256 50256
  50256]
 [  169   251   238   168   252   234  4907 31619   114   230   166   116
    230 35975   112 46695   115  4907   159   227   254   159   227   254
   1279    91   325    79    91    29   220   169   238   230 35975   112
    168   243   120 50256 50256 50256 50256 50256 50256 50256 50256 50256
  50256]
 [  168   117   250   166   113   105 31

In [117]:
EPOCHS = 20
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/20


/home/downtown/miniconda3/envs/fucklms/lib/python3.8/site-packages/keras/src/backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


 26/738 [>.............................] - ETA: 1:06:03 - loss: 6.5758 - accuracy: 0.0574

KeyboardInterrupt: 

## [회고]
학습 돌아가는 것 까지는 확인을 했습니다만,    
... 파라미터를 보시면 아시겠지만 너무 과도하게 나와서    
학습시간도도 말도 안 되게 나와서 일단 임의로 중지시켰습니다.
ㅠㅠ

## 챗봇 테스트

### decoder_inference

In [ ]:
def decoder_inference(sentence):
  sentence = preprocess_sentence(sentence)

  # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
  # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
  # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
  output_sequence = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 인퍼런스 단계
  for i in range(MAX_LENGTH):
    # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
    predictions = model(inputs=[sentence, output_sequence], training=False)
    predictions = predictions[:, -1:, :]

    # 현재 예측한 단어의 정수
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
    # 이 output_sequence는 다시 디코더의 입력이 됩니다.
    output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

  return tf.squeeze(output_sequence, axis=0)


In [ ]:
def sentence_generation(sentence):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
  prediction = decoder_inference(sentence)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('입력 : {}'.format(sentence))
  print('출력 : {}'.format(predicted_sentence))

  return predicted_sentence


### 임의 문장 test 1

기본 코드로 학습시킨 결과  
첫 질문은 반대의 대답을 하고 두 번째 질문은 말이 되는 듯 하다  

[전처리]    
기본 전처리   
두점(?.!,) 앞뒤에 공백 추가   
연속된 공백을 하나의 공백으로 변환   
한글, 영어, 숫자, 기본적인 특수문자(?.!,)만 허용하고 나머지는 공백 처리   

[하이퍼파라미터]   
NUM_LAYERS = 6 # 인코더와 디코더의 층의 개수   
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원   
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수    
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기   
DROPOUT = 0.1 # 드롭아웃의 비율   
EPOCHS = 10   
(교사 강요)   
BATCH_SIZE = 64   
BUFFER_SIZE = 20000

In [ ]:
sentence_generation('흑기사 해주는 짝남.')

In [ ]:
sentence_generation('나 헤어졌어.')

### 임의 문장 test 2

교사 강요에서 BUFFER_SIZE 만 5000으로 수정한 결과  
전 보다는 어느 정도 말이 되긴 하지만 그래도 아직 부족하다  
acc도 0.06 대  
  
[전처리]    
기본 전처리   
두점(?.!,) 앞뒤에 공백 추가   
연속된 공백을 하나의 공백으로 변환   
한글, 영어, 숫자, 기본적인 특수문자(?.!,)만 허용하고 나머지는 공백 처리   

[하이퍼파라미터]   
NUM_LAYERS = 6 # 인코더와 디코더의 층의 개수   
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원   
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수    
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기   
DROPOUT = 0.1 # 드롭아웃의 비율   
EPOCHS = 10   
(교사 강요)   
BATCH_SIZE = 64   
BUFFER_SIZE = 5000

In [ ]:
sentence_generation('흑기사 해주는 짝남.')

In [ ]:
sentence_generation('나 헤어졌어.')

In [ ]:
sentence_generation('혼자 노력하는 연애인 거 같아.')

In [ ]:
sentence_generation('크리스마스인데 만나자고 해도 됨?')

In [ ]:
sentence_generation('12시 땡!')

In [ ]:
sentence_generation('나 심심해.')

In [ ]:
sentence_generation('오늘 저녁 뭐 먹을까?')

### 임의 문장 test 3


lr의 d_model 과 동일하게 128로 맞춤  
드롭아웃도 0.5로 키우고  
교사 강요 batch_size도 16으로 줄였다  
결과는 ..... 처참합니다    
그리고 BUFFER SIZE 관련해서 영진님께서 캐시에 있는 데이터를 버퍼에서 셔플하여 데이터의 편향을 막으려고 사용한 것이므로    
5000개만 데이터를 섞게 되면 편향이 생길 수 있다는 의견을 공유해주셨다    
확실히 결과를 보면 그런 것 같다  
따라서 다음 test에는 다시 20000으로 복귀..   

[전처리]   
기본 전처리   
두점(?.!,) 앞뒤에 공백 추가   
연속된 공백을 하나의 공백으로 변환   
한글, 영어, 숫자, 기본적인 특수문자(?.!,)만 허용하고 나머지는 공백 처리   

[하이퍼파라미터]   
NUM_LAYERS = 6 # 인코더와 디코더의 층의 개수   
D_MODEL = 128 # 인코더와 디코더 내부의 입, 출력의 고정 차원   
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수   
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기   
DROPOUT = 0.5 # 드롭아웃의 비율   
EPOCHS = 10   
(교사 강요)   
BATCH_SIZE = 16   
BUFFER_SIZE = 5000   

In [ ]:
sentence_generation('흑기사 해주는 짝남.')

In [ ]:
sentence_generation('12시 땡!')

### 임의 문장 test 4

In [ ]:
교사 강요 파라미터를 다시 원복하고 에폭만 20으로 늘려봤다   

[전처리]   
기본 전처리   
두점(?.!,) 앞뒤에 공백 추가   
연속된 공백을 하나의 공백으로 변환   
한글, 영어, 숫자, 기본적인 특수문자(?.!,)만 허용하고 나머지는 공백 처리   

[하이퍼파라미터]   
NUM_LAYERS = 6 # 인코더와 디코더의 층의 개수   
D_MODEL = 128 # 인코더와 디코더 내부의 입, 출력의 고정 차원   
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수   
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기   
DROPOUT = 0.5 # 드롭아웃의 비율   
EPOCHS = 20   
(교사 강요)   
BATCH_SIZE = 64   
BUFFER_SIZE = 20000   

In [ ]:
sentence_generation('흑기사 해주는 짝남.')

In [ ]:
sentence_generation('나 헤어졌어.')

In [ ]:
sentence_generation('크리스마스인데 만나자고 해도 됨?')

In [ ]:
sentence_generation('12시 땡!')

In [ ]:
sentence_generation('나 심심해.')

In [ ]:
sentence_generation('오늘 저녁 뭐 먹을까?')